In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('sentitrain.csv')


In [4]:
df2=pd.read_csv('sentitest.csv')

In [5]:
unclean_data=[]
for i in range(0,len(df['text'])):
    unclean_data.append((df['text'].iloc[i].split(),df['airline_sentiment'].iloc[i]))
unclean_data

[(['@SouthwestAir',
   'I',
   'am',
   'scheduled',
   'for',
   'the',
   'morning,',
   '2',
   'days',
   'after',
   'the',
   'fact,',
   'yes..not',
   'sure',
   'why',
   'my',
   'evening',
   'flight',
   'was',
   'the',
   'only',
   'one',
   'Cancelled',
   'Flightled'],
  'negative'),
 (['@SouthwestAir',
   'seeing',
   'your',
   'workers',
   'time',
   'in',
   'and',
   'time',
   'out',
   'going',
   'above',
   'and',
   'beyond',
   'is',
   'why',
   'I',
   'love',
   'flying',
   'with',
   'you',
   'guys.',
   'Thank',
   'you!'],
  'positive'),
 (['@united',
   'Flew',
   'ORD',
   'to',
   'Miami',
   'and',
   'back',
   'and',
   'had',
   'great',
   'crew,',
   'service',
   'on',
   'both',
   'legs.',
   'THANKS'],
  'positive'),
 (['@SouthwestAir', '@dultch97', "that's", 'horse', 'radish', '😤🐴'],
  'negative'),
 (['@united',
   'so',
   'our',
   'flight',
   'into',
   'ORD',
   'was',
   'delayed',
   'because',
   'of',
   'Air',
   'Force',
   

In [6]:
unclean_data_test=[]
for i in range(0,len(df2['text'])):
    unclean_data_test.append((df2['text'].iloc[i].split()))                           
unclean_data_test

[['@AmericanAir',
  'In',
  'car',
  'gng',
  'to',
  'DFW.',
  'Pulled',
  'over',
  '1hr',
  'ago',
  '-',
  'very',
  'icy',
  'roads.',
  'On-hold',
  'with',
  'AA',
  'since',
  '1hr.',
  "Can't",
  'reach',
  'arpt',
  'for',
  'AA2450.',
  'Wat',
  '2',
  'do?'],
 ['@AmericanAir',
  'after',
  'all,',
  'the',
  'plane',
  'didn’t',
  'land',
  'in',
  'identical',
  'or',
  'worse)',
  'conditions',
  'at',
  'GRK',
  'according',
  'to',
  'METARs.'],
 ['@SouthwestAir',
  "can't",
  'believe',
  'how',
  'many',
  'paying',
  'customers',
  'you',
  'left',
  'high',
  'and',
  'dry',
  'with',
  'no',
  'reason',
  'for',
  'flight',
  'Cancelled',
  'Flightlations',
  'Monday',
  'out',
  'of',
  'BDL!',
  'Wow.'],
 ['@USAirways',
  'I',
  'can',
  'legitimately',
  'say',
  'that',
  'I',
  'would',
  'have',
  'rather',
  'driven',
  'cross',
  'country',
  'than',
  'flown',
  'on',
  'US',
  'Airways.'],
 ['@AmericanAir',
  'still',
  'no',
  'response',
  'from',
  'AA

In [7]:
from nltk.stem import WordNetLemmatizer
lemm=WordNetLemmatizer()
from nltk import pos_tag

In [8]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [9]:
from nltk.corpus import stopwords
import string
stops=set(stopwords.words('english'))
punc=list(string.punctuation)
stops.update(punc)

In [10]:
def clean_review(words):
    output_words=[]
    for w in words:
        if w.lower() not in stops:
            pos=(pos_tag([w]))
            clean_word=lemm.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [11]:
documents=[(clean_review(document),category)for document,category in unclean_data]

In [12]:
len(documents)

10980

In [13]:
documents_test=[(clean_review(document))for document in unclean_data_test]

In [14]:
len(documents_test)

3660

In [15]:
all_words=[]
for i in documents:
    all_words+=i[0]
len(all_words)

117488

In [16]:
import nltk 
frequency=nltk.FreqDist(all_words)

In [17]:
common=frequency.most_common(3000)
features=[i[0]for i in common]

In [18]:
def get_feature_dict(words):
    current_features={}
    words_set=set(words)
    for w in features:
        current_features[w]=w in words_set
    return current_features

In [19]:
training_Data=[(get_feature_dict(doc),category)for doc,category in documents]

In [21]:
testing_data=[(get_feature_dict(doc))for doc in documents_test]

In [22]:
from nltk import NaiveBayesClassifier

In [23]:
classifier=NaiveBayesClassifier.train(training_Data)

In [24]:
nltk.classify.accuracy(classifier,training_Data)

0.8375227686703096

In [25]:
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier

In [26]:
svc=SVC()
classifier_sklearn=SklearnClassifier(svc)
classifier_sklearn.train(training_Data)

/Users/vaibhavsinghmakhloga/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


<SklearnClassifier(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))>

In [27]:
nltk.classify.accuracy(classifier_sklearn,training_Data)

0.6239526411657559

In [73]:
predict3=[]
for i in range(len(testing_data)):
    predict3.append(classifier_sklearn.classify(testing_data[i]))

In [74]:
np.savetxt('senti4.csv',predict3,fmt="%s")

In [39]:
predict=[]
for i in range(len(testing_data)):
    predict.append(classifier.classify(testing_data[i]))

In [44]:
predict

['negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',

In [51]:
np.savetxt('senti.csv',predict,fmt="%s")

In [75]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier()
classifier_sklearn1=SklearnClassifier(rfc)
classifier_sklearn1.train(training_Data)
nltk.classify.accuracy(classifier_sklearn1,training_Data)

/Users/vaibhavsinghmakhloga/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9773224043715847

In [77]:
predict4=[]
for i in range(len(testing_data)):
    predict4.append(classifier_sklearn1.classify(testing_data[i]))

In [61]:
predict2

['negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negativ

In [80]:
np.savetxt('senti5.csv',predict4,fmt="%s")

In [64]:
categories =[category for document,category in documents]

In [65]:
text_doc=[" ".join(document) for document,category in documents]

In [67]:
text_doc_test=[" ".join(document) for document in documents_test]

In [69]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec=CountVectorizer(max_features=3000,ngram_range=(1,2))
x_train_features=count_vec.fit_transform(text_doc)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [70]:
x_test_features=count_vec.transform(text_doc_test)

In [71]:
svc2=SVC()
svc2.fit(x_train_features,categories)

/Users/vaibhavsinghmakhloga/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

/Users/vaibhavsinghmakhloga/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:182: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0